# Creating a frequency finder auto-encoder

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import callbacks
import tensorflow.keras as keras
from tensorflow.keras import layers
import umap
from babyplots import Babyplot
#import hdbscan
from matplotlib import pyplot
from models import *
from converters import *
from helpers import *
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

2021-10-06 11:47:42.316944: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-06 11:47:42.316963: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%load_ext autoreload
%autoreload 2

## Get drive path 

In [3]:
drive_path = find_drive_path()
drive_path

UnboundLocalError: local variable 'path' referenced before assignment

## Load Data from drive

In [8]:
try:
    data = pd.read_pickle(f"{drive_path}/Data/Inputs/Synthetisch/Syn_tot.pkl")
except:
    data = pd.read_pickle("../../Data/Syn01_02.pkl")
data.head()

,IID,Pheno,Geno,State
0,0pop1,0,"[2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",pop1healthy
1,1pop1,1,"[0, 2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, ...",pop1sick
2,2pop1,1,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",pop1sick
3,3pop1,1,"[2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 2, ...",pop1sick
4,4pop1,1,"[2, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2, ...",pop1sick


In [9]:
data.tail()

,IID,Pheno,Geno,State
1995,995pop2,1,"[0, 1, 2, 1, 1, 1, 1, 0, 0, 1, 2, 2, 0, 0, 0, ...",pop2sick
1996,996pop2,0,"[1, 1, 1, 0, 1, 2, 0, 0, 0, 1, 2, 1, 1, 0, 2, ...",pop2healthy
1997,997pop2,0,"[0, 2, 1, 1, 2, 0, 1, 2, 0, 2, 1, 1, 1, 1, 1, ...",pop2healthy
1998,998pop2,0,"[2, 2, 0, 0, 2, 1, 1, 0, 2, 0, 1, 1, 1, 0, 0, ...",pop2healthy
1999,999pop2,0,"[0, 0, 0, 2, 2, 0, 1, 1, 2, 1, 0, 2, 1, 1, 0, ...",pop2healthy


## Preprocessing data into tensors 

### Keeping only the minor alleles as 1

In [10]:
X = only_minor(data["Geno"])
X

array([[0., 0., 1., ..., 1., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

### Train test split

In [11]:
split = splitter([X],0.1)
[X_train, X_test] = tensor_converter(split)
X = tensor_converter(X)
X = tf.stack(X, axis = 0)

2021-10-06 11:53:38.876240: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-06 11:53:38.876658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-06 11:53:38.876743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-06 11:53:38.876816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-06 11:53:38.893601: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [12]:
X

<tf.Tensor: shape=(2000, 10000), dtype=float32, numpy=
array([[0., 0., 1., ..., 1., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32)>

## Loading hyper-parameters

In [13]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
PRUNING = 0.5
LEARNING_RATE = 0.01
first_layer = 10

In [14]:
input_len = int(tf.shape(X_train)[1])
ff = frequencyfinder(input_len, first_layer)
ff.build((None, input_len))

In [15]:
output_test = ff(X)

In [16]:
output_test[0]

<tf.Tensor: shape=(10000,), dtype=float32, numpy=
array([1.0857288e-04, 1.0320351e-04, 1.0172599e-04, ..., 9.8352670e-05,
       9.8352670e-05, 9.8352670e-05], dtype=float32)>

In [20]:
from tensorflow_probability.python import layers as tfpl